In [2]:
using OrdinaryDiffEq
using ComponentArrays
using ForwardDiff
using Optimization, OptimizationOptimJL, OptimizationOptimisers
using SciMLSensitivity
using Zygote
using Random
using Plots


In [ ]:

"""
 Here we define a smooth "logistic" helper for transitions
 so that we avoid harsh if-else in the ODEs.
"""
σ(x) = 1.0 / (1.0 + exp(-x))


In [3]:

"""
 Our state vector will be:
   x[1] = product(t)            (in [0,1] ideally)
   x[2] = nCustomersEverTried(t)
   x[3] = totalLostCustomers(t)
   x[4] = accumulatedPnL(t)
   x[5] = valueAddedToCustomers(t)
   x[6] = budget(t)             (we track it so it doesn't go < 0)

 We'll let the "control" be s(t), and define:
    workOnProduct(t)   = σ( s(t) )
    workOnCustomers(t) = 1 - σ( s(t) )

 We want to penalize big changes in s(t),
 so s(t) is approximated by s_k for time-slices k and
 smoothly interpolated if needed. For brevity below,
 we'll treat s(t) as piecewise-constant across intervals
 to illustrate the idea.
"""
function pivo_ode!(dx, x, p, t)
    # Unpack states
    product            = x[1]
    nCustomersEverTried= x[2]
    totalLostCustomers = x[3]
    accumulatedPnL     = x[4]
    valueAdded         = x[5]
    budget             = x[6]

    # Unpack parameters controlling ODE
    productAdvRate                 = p[:productAdvRate]
    marketCap                      = p[:marketCap]
    customerChurnProbIfWorkUnmet   = p[:customerChurnProb]
    initWorkGeneratedByCustomer    = p[:initWorkGen]
    recurringWorkPerCustomer       = p[:recurringWorkPerCustomer]
    marketAcquisitionSpeedMult     = p[:marketAcquisitionSpeedMult]
    salary                         = p[:salary]
    recurringFeePerCustomer        = p[:recurringFee]
    integrationFeeOnboarding       = p[:onboardFee]       # (like "integrationFee")
    productProductivityCap         = p[:prodCap]
    competitorEatsMarketRate       = p[:compRate]         # how quickly others eat the market
    customerAcquisitionSpeed       = p[:acquireSpeed]     # fraction of leftover market we get if max marketing
    penaltyLargeStrategyChange     = p[:penaltyStrategy]
    budgetPenalty                  = p[:budgetPenalty]

    # For demonstration, let's pull a single control "s(t)" from param
    # or from a piecewise function. We'll do a simple approach:
    # s(t) in p[:controls], piecewise index by integer floor(t).
    # (This is a simplistic discretization approach.)
    # Make sure to clamp the index in case t>length.
    s_array = p[:controls]
    idx = min(length(s_array), max(1, Int(floor(t)+1)))
    s_t = s_array[idx]

    # Smoothly define the fraction of team focusing on product vs customers
    wP = σ(s_t)
    wC = 1.0 - wP

    # Some auxiliary definitions from the original story
    totalCurrentCustomers = nCustomersEverTried - totalLostCustomers

    # Reputation formula:  (smooth version, or a direct formula)
    #  rep(t) = marketAcquisitionSpeedMult + (1 - marketAcquisitionSpeedMult)* ...
    #                * totalCurrentCustomers / (nCustomersEverTried + smallEps)
    smallEps = 1e-2
    reputation = marketAcquisitionSpeedMult +
                 (1 - marketAcquisitionSpeedMult)*(totalCurrentCustomers)/(nCustomersEverTried + smallEps)

    # We define costAcquireCustomer as some logistic penalty that goes up
    # if we already have many customers. Just a placeholder:
    costAcquireCustomerMultiplier = 0.4
    costAcquireCustomer = costAcquireCustomerMultiplier * σ( -0.5*(totalCurrentCustomers - 4.0) )

    # Now define rates:

    # 1) d(product)/dt = productAdvRate * (1 - product) * (wP + ???)
    #    We'll just do a small "assist" term so wC also helps but smaller:
    #    (the old code had a "workOnCToWorkOnPTranslation" factor)
    wC2P = 0.15
    dproduct = productAdvRate*(1 - product)*(wP + wC2P*wC)

    # 2) The rate new customers come in. We'll do:
    #    d(nCustomersEverTried)/dt = customerAcquisitionSpeed * reputation
    #                                 * (marketCap - nCustomersEverTried)*product
    #    plus we subtract competitorEatsMarketRate*(marketCap - nCustomersEverTried)
    gainCustomers = customerAcquisitionSpeed * reputation * (marketCap - nCustomersEverTried)*product
    losePotential = competitorEatsMarketRate * (marketCap - nCustomersEverTried)
    dnCustomers = gainCustomers - losePotential
    if dnCustomers < 0 && nCustomersEverTried <= 0
        # clamp to 0 if we have no one
        dnCustomers = 0.0
    end

    # 3) Current work generated by customers = initWork* nCustomersEverTried'(t)
    #     + recurringWorkPerCustomer * totalCurrentCustomers
    #    For smoothness, let's interpret nCustomersEverTried'(t) as gainCustomers only
    #    (the "losePotential" is competitor, not our new signups).
    currentWorkGenerated = initWorkGeneratedByCustomer*(gainCustomers) +
                           recurringWorkPerCustomer*totalCurrentCustomers

    # 4) Value added to customers is d(valueAdded)/dt
    #    = (1 / max(1-product, 1/prodCap))* product * wC
    #    We'll do a smooth variant:  denom = 1-product if product<1,
    #    else 1/productProductivityCap, so let's do a soft mix:
    denom_smooth = (1 - product) + (1/productProductivityCap)
    # Weighted by product in some "product synergy"
    dvalueAdded = (product / denom_smooth)*wC

    # 5) Churn: we lose customers at rate
    #    churnRate = max(0, customerChurnProbIfWorkUnmet*( currentWork - dvalueAdded ) )
    #    We'll do a smooth ReLU: relu(x) ~ log(1+exp(k*x))/k for large k,
    #    but let's just clamp for simplicity:
    unmet = currentWorkGenerated - dvalueAdded
    churnRate = customerChurnProbIfWorkUnmet * max(0, unmet)
    dLost = churnRate

    # 6) Profit & budget flow:
    #    d(accumPnL)/dt = integrationFeeOnboarding * max(dnCustomers,0)
    #                      - costAcquireCustomer*salary*max(dnCustomers,0)
    #                      + recurringFeePerCustomer*totalCurrentCustomers
    #                      - salary*(wP + wC)
    #    Explanation:
    #      - We earn "integrationFeeOnboarding" for each new sign-up
    #      - We pay costAcquireCustomer*salary for each new sign-up
    #      - We earn "recurringFeePerCustomer" from each existing customer
    #      - We pay "salary" for the entire team (some fraction to product, some fraction to customers).
    signups = max(gainCustomers, 0)
    dPnL = integrationFeeOnboarding*signups - costAcquireCustomer*salary*signups +
           recurringFeePerCustomer*totalCurrentCustomers -
           salary*(wP + wC)

    # Budget might be used up by the same negative net flow if it occurs
    # We'll define: d(budget)/dt = - (any net negative from the above)?
    # For simplicity, let's assume budget is decreased by salary and cost
    # but not increased by revenue. We keep it simple. Or do whichever we like:
    # We just track if it goes negative:
    netCost = costAcquireCustomer*salary*signups + salary*(wP + wC)
    dbudget = - netCost
    # in principle we could add positive revenues in too, if we want
    # e.g. + integrationFeeOnboarding*signups + recurringFeePerCustomer*...

    # Fill dx
    dx[1] = dproduct
    dx[2] = dnCustomers
    dx[3] = dLost
    dx[4] = dPnL
    dx[5] = dvalueAdded
    dx[6] = dbudget
end


pivo_ode!

In [4]:

"""
 We'll integrate from t=0..T, and define an objective:
   objective(controls) = final accumulatedPnL
                         - λ1 * sum_of_sqr(Δ controls)
                         - λ2 * big penalty if budget < 0

 We'll do a naive discrete-time approach for the "controls." 
 For a real approach, you'd want continuous-time or
 piecewise polynomial parameterization + colocation or direct adjoint.

 We'll do an example with T=30 steps, each step 1.0 in time,
 so we have 30 control parameters in p[:controls].
"""
function make_params(controls::Vector{Float64})
    # Put everything in a NamedTuple or ComponentArray
    return (
        productAdvRate = 0.1,
        marketCap      = 10.0,
        customerChurnProb = 0.8,
        initWorkGen    = 1.0,
        recurringWorkPerCustomer = 0.1,
        marketAcquisitionSpeedMult = 0.2,
        salary         = 2.0,
        recurringFee   = 0.3,
        onboardFee     = 30.0,
        prodCap        = 5.0,
        compRate       = 0.01,
        acquireSpeed   = 0.3,
        penaltyStrategy= 0.1,   # weight for changes in strategy
        budgetPenalty  = 100.0, # how painful going negative is
        controls       = controls
    )
end


make_params

In [8]:
# One way to fix potential type-inference or NamedTuple construction issues
# is to explicitly construct a NamedTuple. For example:

function make_params(controls::Vector{Float64})
    return NamedTuple{(
       :productAdvRate,
       :marketCap,
       :customerChurnProb,
       :initWorkGen,
       :recurringWorkPerCustomer,
       :marketAcquisitionSpeedMult,
       :salary,
       :recurringFee,
       :onboardFee,
       :prodCap,
       :compRate,
       :acquireSpeed,
       :penaltyStrategy,
       :budgetPenalty,
       :controls
    )}(
       0.1,   # productAdvRate
       10.0,  # marketCap
       0.8,   # customerChurnProb
       1.0,   # initWorkGen
       0.1,   # recurringWorkPerCustomer
       0.2,   # marketAcquisitionSpeedMult
       2.0,   # salary
       0.3,   # recurringFee
       30.0,  # onboardFee
       5.0,   # prodCap
       0.01,  # compRate
       0.3,   # acquireSpeed
       0.1,   # penaltyStrategy
       100.0, # budgetPenalty
       controls
    )
end

make_params (generic function with 1 method)

In [9]:

function budget_penalty(sol)
    # If budget < 0 at any point, we incur big penalty
    # We'll sum something like the integral of negative budget,
    # but let's do a naive approach:
    tvals = sol.t
    bvals = [sol[i][6] for i in 1:length(sol)]
    penalty = 0.0
    for b in bvals
        penalty += max(0, -b)
    end
    return penalty
end


budget_penalty (generic function with 1 method)

In [10]:

"""
 We'll define a function that runs the ODE for a given control vector
 and returns a scalar loss which we want to MINIMIZE.
 So our objective = -(finalPnL) + penaltyForBudget + penaltyForLargeChanges
 We'll do negative final PnL so that we can do standard 'minimization'.
"""
function pivo_loss(controls)
    p = make_params(controls)
    # initial conditions:
    x0 = [0.0, 0.0, 0.0, 0.0, 0.0, 50.0]  # let's say we start with budget=50

    # solve
    prob = ODEProblem(pivo_ode!, x0, (0.0, 30.0), p)
    sol = solve(prob, Tsit5(), saveat=1.0, abstol=1e-8, reltol=1e-8)

    # final PnL is sol[end][4]
    finalPnL = sol[end][4]
    # penalty for changing controls quickly:
    # sum of squared diffs: p[:penaltyStrategy] * Σ (u_k+1 - u_k)^2
    stratpen = 0.0
    for i in 1:length(controls)-1
        stratpen += (controls[i+1] - controls[i])^2
    end
    stratpen *= p[:penaltyStrategy]

    # budget penalty
    budgPen = p[:budgetPenalty] * budget_penalty(sol)

    # define total "cost" we want to MINIMIZE
    # negative profit + strategy penalty + budget penalty
    # (If we want to maximize profit, we do cost = -finalPnL)
    return -finalPnL + stratpen + budgPen
end


pivo_loss

In [11]:

# We'll now set up an optimization:
# we have 31 discrete time points from t=0..30, let's define 31 controls
# or just 30, whatever. We'll guess random initial controls in [-1,1].
rng = Random.default_rng()
N_controls = 30
controls_init = [rand(rng)*2-1 for _ in 1:N_controls]

loss_func = OptimizationFunction((u,p)->pivo_loss(u), 
                                Optimization.AutoForwardDiff())

optprob = OptimizationProblem(loss_func, controls_init)

res_adam = solve(optprob, 
                 OptimizationOptimisers.Adam(0.05), 
                 maxiters=200)

# Then refine with e.g. BFGS
optprob2 = OptimizationProblem(loss_func, res_adam.u)
res_bfgs = solve(optprob2, 
                 OptimizationOptimJL.BFGS(); 
                 maxiters=300)

println("Result with BFGS:")
println(" Optimal controls = ", res_bfgs.u)
println(" Objective value = ", res_bfgs.minimizer)

# We can also visualize the solution
best_controls = res_bfgs.u
p_best = make_params(best_controls)
x0 = [0.0, 0.0, 0.0, 0.0, 0.0, 50.0]
prob_best = ODEProblem(pivo_ode!, x0, (0.0, 30.0), p_best)
sol_best = solve(prob_best, Tsit5(), saveat=0.5)

tplot = sol_best.t
product_vals   = [sol_best[i][1] for i in 1:length(tplot)]
custTried_vals = [sol_best[i][2] for i in 1:length(tplot)]
lost_vals      = [sol_best[i][3] for i in 1:length(tplot)]
pnl_vals       = [sol_best[i][4] for i in 1:length(tplot)]
budget_vals    = [sol_best[i][6] for i in 1:length(tplot)]

p1 = plot(tplot, product_vals, label="product(t)", title="Product Maturity")
p2 = plot(tplot, custTried_vals, label="nCustomersEverTried(t)", title="Customers Tried")
p3 = plot(tplot, lost_vals, label="Lost Customers(t)", title="Lost Customers")
p4 = plot(tplot, pnl_vals, label="accumPnL(t)", title="Accumulated PnL")
p5 = plot(tplot, budget_vals, label="budget(t)", title="Budget")

plot(p1, p2, p3, p4, p5, layout=(3,2), size=(1000,800))

MethodError: MethodError: no method matching make_params(::Vector{ForwardDiff.Dual{ForwardDiff.Tag{DifferentiationInterface.FixTail{var"#27#28", Tuple{SciMLBase.NullParameters}}, Float64}, Float64, 10}})
The function `make_params` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  make_params(!Matched::Vector{Float64})
   @ Main ~/Documents/GitHub/ML-notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X11sZmlsZQ==.jl:4
